In [1]:
import pandas as pd
from processor.processor import Processor as p
from modeler.modeler import Modeler as m
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from tqdm import tqdm
from database.market import Market

In [4]:
market = Market()

In [5]:
market.connect()
sp5 = market.retrieve("sp500")
prices = market.retrieve("prices")
market.disconnect()

In [6]:
strategies = ["rolling","window"]
rollings = [7,14,30,100]
signals = [0.05,0.1,0.15,30]
reqs = [0.05,0.1,0.15,30]
values = [True,False]

In [7]:
parameters = []
for strategy in strategies:
    for rolling in rollings:
        for signal in signals:
            for req in reqs:
                for value in values:
                    parameters.append({"strategy":strategy,
                                      "rolling":rolling,
                                      "signal":signal,
                                      "req":req,
                                      "value":value})

In [8]:
print(len(parameters))

320


In [9]:
prices = p.column_date_processing(prices)

In [11]:
transformed = []
for ticker in tqdm(sp5["Symbol"].unique()):
    ticker_data = prices[prices["ticker"]==ticker].sort_values("date")
    for rolling in rollings:
        ticker_data[f"rolling_{rolling}"] = ticker_data["adjclose"].rolling(window=rolling).mean()
        ticker_data[f"window_{rolling}"] = (ticker_data["adjclose"] - ticker_data["adjclose"].shift(rolling)) / ticker_data["adjclose"].shift(rolling)
        transformed.append(ticker_data)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 497/497 [01:25<00:00,  5.79it/s]


In [12]:
sim = pd.concat(transformed)

In [31]:
start = datetime(2016,1,1)
end = datetime(2022,1,1)
trades = []
for parameter in tqdm(parameters):
    iterration_sim = sim.copy()
    strategy = parameter["strategy"]
    rolling = parameter["rolling"]
    iterration_sim["delta"] = (iterration_sim["adjclose"] - iterration_sim[f"{strategy}_{rolling}"]) / iterration_sim[f"{strategy}_{rolling}"]
    if parameter["value"] == False:
        iterration_sim["delta"] = iterration_sim["delta"] * -1
    for position in range(5):
        date = start
        while date < end:
            current = iterration_sim[iterration_sim["date"]==date]
            if current.index.size < 1:
                date = date + timedelta(days=1)
            else:
                offerings = current.sort_values("delta",ascending=False)
                if offerings.index.size > position:
                    offering = offerings.iloc[position]
                    if offering["delta"].item() >= parameter["signal"]:
                        ticker = offering["ticker"]
                        trade = offering[["date","adjclose","ticker"]].to_dict()
                        exits = iterration_sim[(iterration_sim["date"]>date) & (iterration_sim["ticker"]==ticker)]
                        exits["gain"] = (exits["adjclose"] - trade["adjclose"]) / trade["adjclose"]
                        profits = exits[exits["gain"]>=parameter["req"]]
                        if profits.index.size < 1:
                            exit = exits.iloc[-1]
                            trade["sell_price"] = exit["adjclose"]
                        else:
                            exit = profits.iloc[0]
                            trade["sell_price"] = trade["adjclose"] * (1+parameter["req"])
                        trade["sell_date"] = exit["date"]
                        for key in parameter.keys():
                            trade[key] = parameter[key]
                        trade["delta"] = (trade["sell_price"] - trade["adjclose"]) / trade["adjclose"]
                        trade["position"] = position
                        trades.append(trade)
                        date = exit["date"] + timedelta(days=1)
                    else:
                        date = date + timedelta(days=1)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 320/320 [9:15:49<00:00, 104.22s/it]


In [35]:
t = pd.DataFrame(trades)

In [47]:
analysis = []
fee_rate = 0.0
seed_funding = 100
for parameter in parameters:
    iteration = t.copy()
    for key in parameter.keys():
        iteration = iteration[iteration[key]==parameter[key]]
    if iteration.index.size > 0:
        iteration.sort_values("date",inplace=True)
        initial = seed_funding
        fee = 0
        for position in range(5):
            position_initial = initial / 5
            position_trade = iteration[iteration["position"]==position]
            if position_trade.index.size > 0:
                for delta in position_trade["delta"]:
                    cash_inflow = position_initial * (delta)
                    if delta > 0:
                        trade_fee = cash_inflow * fee_rate
                        new_cash_inflow = cash_inflow - trade_fee
                        position_initial += new_cash_inflow
                        position_initial += trade_fee
                    else:
                        position_initial += cash_inflow
                new_dictionary = {}
                for key in parameter.keys():
                    new_dictionary[key] = parameter[key]
                new_dictionary["pv"] = position_initial
                new_dictionary["position"] = position
                new_dictionary["fee"] = fee
                analysis.append(new_dictionary)

In [48]:
a = pd.DataFrame(analysis)
a.sort_values("pv",ascending=False).head(20)

,strategy,rolling,signal,req,value,pv,position,fee
115,rolling,7,0.15,30.0,False,1673.479907,0,0
119,rolling,7,0.15,30.0,False,1673.479907,4,0
118,rolling,7,0.15,30.0,False,1673.479907,3,0
117,rolling,7,0.15,30.0,False,1673.479907,2,0
116,rolling,7,0.15,30.0,False,1673.479907,1,0
235,rolling,14,0.15,30.0,False,951.706148,0,0
236,rolling,14,0.15,30.0,False,951.706148,1,0
237,rolling,14,0.15,30.0,False,951.706148,2,0
238,rolling,14,0.15,30.0,False,951.706148,3,0
239,rolling,14,0.15,30.0,False,951.706148,4,0


In [49]:
whut = a.pivot_table(index=parameter.keys(),values="pv",columns="position").reset_index()
whut.fillna(100/5,inplace=True)
whut["pv"] = [sum([row[1][i] for i in range(5)]) for row in whut.iterrows()]

In [60]:
whut.sort_values("pv",ascending=False).head(20)

position,strategy,rolling,signal,req,value,0,1,2,3,4,pv
22,rolling,7,0.15,30.00,False,1673.479907,1673.479907,1673.479907,1673.479907,1673.479907,8367.399534
46,rolling,14,0.15,30.00,False,951.706148,951.706148,951.706148,951.706148,951.706148,4758.530742
63,rolling,30,0.10,30.00,True,613.070417,613.070417,613.070417,613.070417,613.070417,3065.352084
71,rolling,30,0.15,30.00,True,613.070417,613.070417,613.070417,613.070417,613.070417,3065.352084
55,rolling,30,0.05,30.00,True,613.070417,613.070417,613.070417,613.070417,613.070417,3065.352084
119,rolling,100,0.15,30.00,True,425.775510,425.775510,425.775510,425.775510,425.775510,2128.877551
79,rolling,90,0.05,30.00,True,425.775510,425.775510,425.775510,425.775510,425.775510,2128.877551
87,rolling,90,0.10,30.00,True,425.775510,425.775510,425.775510,425.775510,425.775510,2128.877551
95,rolling,90,0.15,30.00,True,425.775510,425.775510,425.775510,425.775510,425.775510,2128.877551
103,rolling,100,0.05,30.00,True,425.775510,425.775510,425.775510,425.775510,425.775510,2128.877551
